In [ ]:
# import libraries
%matplotlib inline

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import cm
from shapely.geometry import Polygon
from shapely.ops import cascaded_union
import geopandas as gpd


from sentinelhub import FisRequest, BBox, Geometry, CRS, WcsRequest, CustomUrlParam, \
    DataCollection, HistogramType, bbox_to_dimensions
from sentinelhub.time_utils import iso_to_datetime
from sentinelhub import DataCollection

In [ ]:
# sentinel hub configurations
from sentinelhub import SHConfig


INSTANCE_ID = ''  # In case you put instance ID into configuration file you can leave this unchanged

if INSTANCE_ID:
    config = SHConfig()
    config.instance_id = INSTANCE_ID
else:
    config = None

In [ ]:
def fis_data_to_dataframe(fis_data):
    """ Creates a DataFrame from list of FIS responses
    """
    COLUMNS = ['channel', 'date', 'min', 'max', 'mean', 'stDev']
    data = []

    for fis_response in fis_data:
        for channel, channel_stats in fis_response.items():
            for stat in channel_stats:
                row = [int(channel[1:]), iso_to_datetime(stat['date'])]

                for column in COLUMNS[2:]:
                    row.append(stat['basicStats'][column])

                data.append(row)

    return pd.DataFrame(data, columns=COLUMNS).sort_values(['channel', 'date'])

In [ ]:
# convert to clean df ready for analysis
def clean_df (json_stats):
    raw_df_stats = fis_data_to_dataframe(json_stats).reset_index(drop=True)
    index_df = raw_df_stats[raw_df_stats.channel ==0]
    cloud_df = raw_df_stats[raw_df_stats.channel ==1]
    cloud_df = cloud_df.drop(['channel'], axis=1).add_suffix('_clm')
    merged_df = pd.merge(index_df, cloud_df, left_on ='date', right_on = 'date_clm', how='outer')
    clean_merged_df = merged_df.drop(['channel', 'date_clm'], axis=1)
    return clean_merged_df

In [ ]:
# read single_region geojson in geopandas
single_region = gpd.read_file("ADD YOUR AREA OF INTEREST (.shp or .geojson ) HERE")
single_region_geometry = Geometry(single_region.geometry.values[0], crs=CRS.WGS84)

# for the next interation use cascaded_union to combine 
# all polygons in the multipolygon by using all the external point as their boundaries

In [ ]:
# initialize request to obtain basic stats and histogram values
# warning if the timeline is to long there if error timeout at their server
single_region_geometry_n = FisRequest(
    data_collection=DataCollection.SENTINEL5P,
    layer='NO2',
    geometry_list=[single_region_geometry],
    time=('2020-06-20', '2020-06-22'),
    resolution='1000m',
    config=config
)

single_region_geometry_stats_n = single_region_geometry_n.get_data()

In [ ]:
# request will return json output
single_region_geometry_stats_n

In [ ]:
single_region_stats_clean = clean_df(single_region_geometry_stats_n)
single_region_stats_clean

In [ ]:
single_region_stats_clean.to_csv('regional_no2_results/ARMM_region_stats_clean.csv', index=False)

## END